## Infos ( à supprimer ensuite)

## Variables à ajouter issues des recherches biblio ( à supprimer ensuite)

Variable(s): 

- Population des pays représentés 
- Variable indiquant si le pays est communiste ou pas 
- Variable indiquant si le pays est stable ou pas 
- Variable indiquant si le pays est émergeant ou pas ( otherwise pauvre , riche à peaufiner par la suite) 
- Variable indiquant si le pays a la culture du sport ou pas. 
- Variable indiquant le PIB par habitant 
- Variable indiquant les financements alloués aux sports (olympiques ou pas)


Il faut créer le train et le test avec des données 2024 , il faut les construire , il faut aussi savoir traiter le fait que cette fois ci la Russie ne sera pas disponible .
Faire indice : pays perturbé 1 avec Russie par exemple ou bien on enlève la Russie  (contexte géopolitique simple, on tah les années où j'ai eu un pb )

But c'est de prédire le nombre de médailles pour chaque pays , indicateur simple 
EXTRA POUR AVOIR 20 : Attribuer un poid aux médailles -> pour potentiellement savoir le nb d'or , argent , bronze

Attention au COVID 19 , influence public , web scrapping sur 30 ans de tous les pays qui ont organisé les jeux 
Attention aux pays pas autorisés à concourir 
Pays en guerre qui envoient moins d'athlètes 
Ukraine 
Palestine 
Iles caiman -> doivent être rattachées à un pays 

JO hiver 
JO été Corrélation 
Apparition des femmes dans les JO ( échantillonnage ) 

Culture du sport : Comment la créer ? 
Budget alloué aux sports (investissement , infrastructures ... ) 
Nb de professionnels 
Nb de licencés 
Nb de sports olympiques disponibles à l'école 

Variable : Pays Communiste ou pas 

Croissance pays économique 

Athlète à partir de 25 ans il n'a plus le niveau : on commence à partir de 2000 , JO Sydney 

Indice dvp des pays à recevoir 

Variable pays organisateur 

Sport qui ont des gens avec un âge bien définie (exemple skate Japonais c'est que des jeunes <= 18 ans)

Dataset coupes mondes de discipline  
    
Variable Note selon l'âge 

Athlète qui sont comptés comme participant à deux pays 

Variable : religion majoritaire pays 

Espérance de vie par pays 
Variable pays dvp ou pas 


Recupérer données JO 2018 et 2021

Variable Poids par Médailles : échelloner les JO de 1 à 10 pour 
Variable Poids par année : 

# Introduction  (Business Understanding) 

## Détermination objectifs métier, analytique du projet 

## Identifier les contraintes, limites et risque du projet

## Chargement des packages si besoin 

In [51]:
%load_ext ipython_memory_usage 
!pip install ydata_profiling

from ydata_profiling import ProfileReport

import pandas as pd
import numpy as np
import ipython_memory_usage.ipython_memory_usage as imu
from describe_csv import *  # charge le fichier describe_csv.py par chunk de 10 000 (10**4)

import math
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import scipy as sp
import scipy.stats
from pivottablejs import pivot_ui
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn import metrics
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline, make_union
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier


from bs4 import BeautifulSoup
import requests
import re

imu.start_watching_memory() 

The ipython_memory_usage extension is already loaded. To reload it, use:
  %reload_ext ipython_memory_usage
In [51] used -21.3 MiB RAM in 3.09s (system mean cpu 40%, single max cpu 100%), peaked 21.8 MiB above final usage, current RAM usage now 413.5 MiB


# CODE  

## Bases de données disponibles 

In [52]:
df_athlete=pd.read_csv('data/athlete_events.csv')
df_region=pd.read_csv('data/noc_regions.csv')
df_dictionnary=pd.read_csv('data/worldcities.csv', sep=';')

In [52] used 79.1 MiB RAM in 0.54s (system mean cpu 47%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 492.6 MiB


## Récupération de données externes
Pour améliorer le modèle, nous avons pris la décision d'aller scrapper des données externes (pays organisateurs), et également de récupérer des datasets sur Kaggle (données de Tokyo 2020)

#### Pays organisateurs

In [ ]:
# TODO: Changer le nom des variables
url='https://www.topendsports.com/events/summer/hosts/list.htm'
response=requests.get(url)
url_2='https://en.wikipedia.org/wiki/List_of_Olympic_Games_host_cities'

response_2=requests.get(url_2)

soup_2=BeautifulSoup(response_2.text,'html.parser')

In [ ]:
# TODO: Changer le nom des variables

# Récupération des éléments de la page
L_pays=[]
L_annees=[]
elements=soup_2.findAll('span',class_='datasortkey')

for element in elements:
    pays=element.find('a').text
    L_pays.append(pays)

L_pays=L_pays[42:58]

In [ ]:
# Itération des années
L_annees_2=[2000 + i  for i in range(0,34,2)]
L_annees_2.pop(-2)

L_annees_2

In [ ]:
# On lie les pays et les années

L_pays_annee=[]
for i in range(len(L_pays)):
    L_pays_annee.append((L_pays[i],L_annees_2[i]))

df_hote=pd.DataFrame(L_pays_annee,columns=['Host_country','Year'])

df_hote

#### Pays avec un passé communiste

In [78]:
url_soviet='https://en.wikipedia.org/wiki/Post-Soviet_states'
response_soviet=requests.get(url_soviet)
soup_soviet=BeautifulSoup(response_soviet.text,'html.parser')

L_pays_soviet=[]

pays_soviet=soup_soviet.findAll('b')
#print(pays_soviet)
for pays in pays_soviet:
    b=pays.find('a')
    if b is not None:
        L_pays_soviet.append(b.text)
L_pays_soviet=L_pays_soviet[:15]

complete_urss_regime=lambda x : 1 if x in L_pays_soviet else 0

NameError: name 'requests' is not defined

In [78] used 30.4 MiB RAM in 0.15s (system mean cpu 37%, single max cpu 81%), peaked 0.0 MiB above final usage, current RAM usage now 485.6 MiB


#### Scrap population et GDP

In [ ]:
url_pop_gpa='https://www.worldometers.info/gdp/gdp-by-country/'
response_pop_gpa=requests.get(url_pop_gpa)
soup_pop_gpa=BeautifulSoup(response_pop_gpa.text,'html.parser')

L_pays=[]

all_countries=soup_pop_gpa.findAll('td')

for country in all_countries:
    a=country.find('a')
    if a is not None:
        L_pays.append(a.text)
len(L_pays)

In [ ]:
all_info=soup_pop_gpa.findAll('td',attrs={'style':'font-weight: bold; text-align:right;'})

numbers=[]
for i,info in enumerate(all_info):
    b=re.findall(r'\d+\b', info.text)
    b=''.join(b)
    numbers.append(b)
   
population=[]
gpd_per_habitant=[]
PIB_liste=[]
for i in range(0,len(numbers),6):
    PIB_liste.append(int(numbers[i]))
for i in range(3,len(numbers),6):
    population.append(int(numbers[i]))
for i in range(4,len(numbers),6):
    gpd_per_habitant.append(int(numbers[i]))

print(len(PIB_liste))
PIB_liste

In [ ]:
# On replace les pays qui n'ont pas le même nom
df_pop_gpd=pd.DataFrame({'Country':L_pays,'PIB':PIB_liste,'Population':population,'PIB/habitant':gpd_per_habitant})

df_pop_gpd['Country']=df_pop_gpd['Country'].replace('Trinidad and Tobago','Trinidad')
df_pop_gpd['Country']=df_pop_gpd['Country'].replace('State of Palestine','Palestine, Occupied Territories')
df_pop_gpd['Country']=df_pop_gpd['Country'].replace('Singapore','Singapour')
df_pop_gpd['Country']=df_pop_gpd['Country'].replace('Czech Republic (Czechia)','Czech Republic')
df_pop_gpd['Country']=df_pop_gpd['Country'].replace("Côte d'Ivoire" ,"Cote d'Ivoire")

df_pop_gpd

In [ ]:
# Pays qui manquaient sur le site scrappé, données récupérées manuellement
df_pop_gpd=pd.concat([df_pop_gpd,pd.DataFrame([['Taiwan',762670000000,23923276,44460],
                                               ['Cook Islands',220000000,17044,13558],
                                               ['Eritrea',1960000000,3684000,1067],
                                               ['Kosovo',9410000000,1760000,5340],
                                               ['Lebanon',23130000000,5425542,4136],
                                               ['Netherlands Antilles',3810000000,27148,11400],
                                               ['South Sudan',12000000000,11000000,467], 
                                               ['American Samoa',871000000,43914,19673],
                                              ['Aruba',3545000000,106445,333000],
                                              ['Bermuda',7645000000,63532,118774],
                                               ['British Virgin Islands',1751000000,31707,34246],
                                               ['Virgin Islands',4440000000,105413,41974],
                                               ['Cayman Islands',6845000000,69310,99624],
                                               ['Cuba',633400000000,11179471,56495],
                                               ['Guam',6123000000,173825,35904],
                                               ['North Korea',48000000000,26070000,33719],
                                               ['Liechtenstein',7710000000,39039,197504],
                                               ['Monaco',8784000000,39050,240862],
                                               ['Puerto Rico',113400000000,3260314,35208],
                                               ['San Marino',1855000000,33642,53000],
                                               ['Syria',8970000000,22130000,420],
                                               ['Venezuela',92000000000,29254044,2624],
                                               ['Virgin Islands*',4444000000,105413,41976],
                                               ['Afghanistan',14583135237,42239000,363.73],
                                               ['Bhutan',2539551327,782455,3560],
                                               ['Brunei',16760000000,449000,37152],
                                               ['Burma',62260000000,54180000,1149],
                                               ['Cape Verde',2227000000,593000,3754],
                                               ['Djibouti',3515000000,1121000,3136],
                                               ["East Timor (Timor-Leste)",3205000000,1341000,2389],
                                               ['Macedonia',13560000000,2058000,6591],
                                               ['Nauru',151600000,12668,11970],
                                               ['Palau',232000000,18055,12921],
                                               ['Saint Kitts and Nevis',965000000,47000,20262],
                                               ['Saint Vincent and the Grenadines',948000000,9123,9125],
                                               ['Sao Tome and Principe',542000000,227380,2386],
                                               ['Somalia',10042000000,17000000,592],
                                               ['Swaziland',4791000000,1202000,3986],
                                               ['Tonga',469000000,106858,4425],
                                               ['Turkmenistan',56540000000,6431000,8792],
                                               ['Yemen',21160000000,33700000,650]
                                               ],
                                             columns=['Country','PIB','Population','PIB/habitant'])],ignore_index=True)

In [ ]:
## Dictionnaire pour remplir les valeurs NaN de Pop et PIB sans changer les valeurs de pop et PIB per habitant précédentes
dictio_pop_gpd=dict(zip(df_pop_gpd['Country'],df_pop_gpd['PIB']))

dictio_pop_gpd

#### JO de Tokyo

## Optimisation des datasets
   Dans un souci d'optimisation de l'espace mémoire occupé, nous allons faire en sorte d'optimiser les différents dataframe que nous utilisons. 
Pour mener à bien ceci, nous allons des fonctions contenues dans `describe_csv.py`, en les améliorant pour automatiser le processus.

In [53]:
#### Code Nathan à ajouter

In [53] used 0.1 MiB RAM in 0.11s (system mean cpu 46%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 492.7 MiB


## Nettoyage

In [54]:
## Ajouter le code FLO

In [54] used 0.0 MiB RAM in 0.10s (system mean cpu 37%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 492.7 MiB


### Dataframe dictionnary

In [55]:
df_dictionnary=pd.read_csv('data/dictionary.csv')

df_dictionnary=df_dictionnary.rename(columns={'GDP per Capita':'PIB/habitant','Code':'NOC'})

df_dictionnary.isna().sum()

Country          0
NOC              0
Population       5
PIB/habitant    25
dtype: int64

In [55] used 0.4 MiB RAM in 0.11s (system mean cpu 69%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 493.1 MiB


### Dataframe région

In [56]:
df_region=df_region.rename(columns={'region':'Country'})

In [56] used 0.0 MiB RAM in 0.11s (system mean cpu 35%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 493.2 MiB


Ajout des îles Tuvalu

In [57]:
df_region[df_region['Country'].isnull()]

,NOC,Country,notes
168,ROT,NaN,Refugee Olympic Team
208,TUV,NaN,Tuvalu
213,UNK,NaN,Unknown


In [57] used 0.1 MiB RAM in 0.11s (system mean cpu 39%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 493.3 MiB


In [58]:
df_region=df_region.query(" NOC != 'ROT' and NOC != 'UNK' ")

df_region['Country']=df_region['Country'].fillna('Tuvalu')

In [58] used 1.0 MiB RAM in 0.11s (system mean cpu 31%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 494.3 MiB


In [59]:
df_region

,NOC,Country,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN
...,...,...,...
225,YEM,Yemen,NaN
226,YMD,Yemen,South Yemen
227,YUG,Serbia,Yugoslavia
228,ZAM,Zambia,NaN


In [59] used 0.0 MiB RAM in 0.11s (system mean cpu 68%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 494.4 MiB


## Merge des dataframes et traitement des valeurs nulles

### Opérations sur les pays

In [60]:
df_region

,NOC,Country,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN
...,...,...,...
225,YEM,Yemen,NaN
226,YMD,Yemen,South Yemen
227,YUG,Serbia,Yugoslavia
228,ZAM,Zambia,NaN


In [60] used 0.0 MiB RAM in 0.11s (system mean cpu 30%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 494.4 MiB


In [61]:
df_complete=pd.merge(pd.merge(df_athlete,df_region,on='NOC',how='left'),df_dictionnary,on='NOC',how='left')

# On prend seulement les JO d'été
df_complete=df_complete[df_complete['Season']=='Summer']
df_complete=df_complete[df_complete['Year'] >= 2000]
df_complete

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Country_x,notes,Country_y,Population,PIB/habitant
0,A Dijiang,M,24.0,180.00,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN,China,1.371220e+09,8027.683810
1,A Lamusi,M,23.0,170.00,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN,China,1.371220e+09,8027.683810
2,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN,Denmark,5.676002e+06,51989.293471
3,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN,Denmark,5.676002e+06,51989.293471
4,Christine Jacoba Aaftink,F,21.0,185.00,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands,NaN,Netherlands,1.693652e+07,44299.768085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282985,ZOU Jingyuan,M,23.0,1.58,NaN,People's Republic of China,CHN,Tokyo 2020,2020,Summer,Tokyo,Artistic Gymnastics,Men's Parallel Bars,Gold,China,NaN,China,1.371220e+09,8027.683810
282986,ZUBIMENDI Martin,M,22.0,1.80,NaN,Spain,ESP,Tokyo 2020,2020,Summer,Tokyo,Football,Men,Silver,Spain,NaN,Spain,4.641827e+07,25831.582305
282987,ZUEV Alexander,M,25.0,1.93,NaN,ROC,ROC,Tokyo 2020,2020,Summer,Tokyo,3x3 Basketball,Men,Silver,NaN,NaN,NaN,NaN,NaN
282988,ZVEREV Alexander,M,24.0,1.98,NaN,Germany,GER,Tokyo 2020,2020,Summer,Tokyo,Tennis,Men's Singles,Gold,Germany,NaN,Germany,8.141314e+07,41313.313995


In [61] used 57.3 MiB RAM in 0.30s (system mean cpu 34%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 551.7 MiB


In [71]:
# Attribution des pays en fonction des NOC 
df_complete=pd.merge(df_complete,df_region,on='NOC',how='left')
df_complete['Country']=df_complete['Country_y'].fillna(df_complete['Country_x'])
df_complete= df_complete.drop(['Country_x', 'Country_y', 'notes_x','notes_y'], axis=1)

In [71] used 62.9 MiB RAM in 0.29s (system mean cpu 43%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 254.8 MiB


In [64]:
# On retire les NOC (réfugiés)
df_complete=df_complete.query(" NOC !='ROC' ")

In [64] used 53.3 MiB RAM in 0.21s (system mean cpu 37%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 627.6 MiB


In [65]:
# Analyse des valeurs nulles 
df_complete.isna().sum()

# 156 sports où on sait pas ce que c'est .... et des country (26 221) qui ne sont pas attribués non plus 

Name                 0
Sex                156
Age               9630
Height           67144
Weight           74388
Team                 0
NOC                  0
Games                0
Year                 0
Season               0
City                 0
Sport              156
Event             9278
Medal           240611
notes_x         277512
Population       27595
PIB/habitant     35921
Country            421
notes_y         277512
dtype: int64

In [65] used -194.1 MiB RAM in 0.47s (system mean cpu 38%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 274.7 MiB


In [67]:
# Attribution des 
liste_pays=df_region['Country'].tolist()
liste_NOC=df_region['NOC'].tolist()

dictio_NOC_pays=dict(zip(liste_NOC,liste_pays))

dictio_NOC_pays['SGP']='Singapour'
dictio_NOC_pays['LBN']='Liban'

df_complete['Country']=df_complete['Country'].fillna(df_complete['NOC'].map(dictio_NOC_pays))


In [67] used 1.6 MiB RAM in 0.18s (system mean cpu 37%, single max cpu 96%), peaked 0.0 MiB above final usage, current RAM usage now 159.3 MiB


/var/folders/38/1vcxjcxx0z75mdh8whh4pw6r0000gn/T/ipykernel_59313/1588779046.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_complete['Country']=df_complete['Country'].fillna(df_complete['NOC'].map(dictio_NOC_pays))


In [69]:
# Il y a quatre NOC qui ne correspondent à aucun pays dans notre EOR : réfugiés , ROT réfugiés aussi , LBN c'est le Liban (Lebanon en anglais )
df_complete=df_complete.query(" NOC !='ROT' and NOC !='EOR' ")

In [69] used 49.3 MiB RAM in 0.20s (system mean cpu 38%, single max cpu 97%), peaked 0.0 MiB above final usage, current RAM usage now 186.9 MiB


In [ ]:
# On ajoute les pays organisateurs pour chaque ligne scrappée précédemment
df_complete=pd.merge(df_complete,df_hote,on='Year',how='left')

In [ ]:
# On ajoute une colonne booléenne (l'athlète participe aux jeux dans son propre pays)
compare_col=lambda x : 1 if x['Country']==x['Host_country'] else 0 

df_complete['Participation_own_country']=df_complete.apply(compare_col,axis=1)
df_complete[df_complete['Participation_own_country']==1]

In [ ]:
# On récupère les pays communistes scrappés précédemment
df_complete['Soviet_past']=df_complete['Country'].apply(complete_urss_regime)

In [ ]:
# On renomme les pays pour pouvoir merge avec les données scrappées 
# Changement de noms pour les pays

df_complete['Country']=df_complete['Country'].replace('Puerto Rico*','Puerto Rico')
df_complete['Country']=df_complete['Country'].replace('Hong Kong*','Hong Kong')
df_complete['Country']=df_complete['Country'].replace('Korea, South','South Korea')
df_complete['Country']=df_complete['Country'].replace('Korea, North','South Korea')
df_complete['Country']=df_complete['Country'].replace('Liban','Lebanon')
df_complete['Country']=df_complete['Country'].replace('Congo, Dem Rep','DR Congo')
df_complete['Country']=df_complete['Country'].replace('Cayman Islands*','Cayman Islands')
df_complete['Country']=df_complete['Country'].replace('Bermuda*','Bermuda')
df_complete['Country']=df_complete['Country'].replace('Virgin Islands*','Virgin Islands')
df_complete['Country']=df_complete['Country'].replace('Aruba*','Aruba')
df_complete['Country']=df_complete['Country'].replace('American Samoa*','American Samoa')
df_complete['Country']=df_complete['Country'].replace('Netherlands Antilles*','Netherlands Antilles')

# URSS 
df_complete.loc[(df_complete['Team'] == 'Soviet Union') & (df_complete['NOC'] == 'URS'), ['Team', 'NOC']] = ['Russia', 'RUS'] # URSS période 1952-1988

# Réfugiés
df_complete.loc[(df_complete['Team'] == 'ROC') & (df_complete['NOC'] == 'ROC'), ['Team', 'NOC']] = ['Germany', 'RUS'] # JO 2020

# GERMANY
df_complete.loc[(df_complete['Team'] == 'East Germany') & (df_complete['NOC'] == 'GDR'), ['Team', 'NOC']] = ['Germany', 'GER']

# CHINA
df_complete.loc[df_complete['Team'] == "People's Republic of China", ['Team', 'NOC']] = ['China', 'CHN']

# UNITED STATES
df_complete.loc[df_complete['Team'] == "United States of America", ['Team', 'NOC']] = ['United States', 'USA']

In [ ]:
# Normalisation des pays. Certains sont écrits avec des suffixes (Ex: Brazil-1, Brazil-2)
replacement_dict = {}

old_countries_to_process = df_complete.Team.unique()

for i in old_countries_to_process:
    replacement_dict[i] = re.sub(r'[\d-]', '', i)


final_df = df_complete.replace({"Team": replacement_dict})

In [ ]:
df_test_na_pop=df_complete[df_complete['Population'].isna()]

# On merge les données de df_test_na_pop et df_complete
df_test_na_pop=pd.merge(df_test_na_pop,df_pop_gpd,on='Country',how='left')
df_test_na_pop.isna().sum()

In [ ]:
# On nettoie les colonnes résiduelles
df_test_na_pop=df_test_na_pop.drop(['Population_x'],axis=1)
df_test_na_pop=df_test_na_pop.drop(['PIB/habitant_x'],axis=1)

# On renomme les colonnes
df_test_na_pop=df_test_na_pop.rename(columns={'Population_y':'Population'})
df_test_na_pop=df_test_na_pop.rename(columns={'PIB/habitant_y':'PIB/habitant'})

# On nettoie les données nulles dans la colonne Population
df_test_na_pop=df_test_na_pop.dropna(subset='Population')
df_complete=df_complete.dropna(subset='Population')

df_complete=pd.concat([df_complete,df_test_na_pop],ignore_index=True)

### Opérations sur le PIB

In [ ]:
# Traitement des données nulles
df_test_na_pib=df_complete[df_complete['PIB/habitant'].isna()]
df_complete=df_complete.dropna(subset='PIB/habitant')

# On merge avec les données scrappées
df_test_na_pib=pd.merge(df_test_na_pib,df_pop_gpd,on='Country',how='left')

# On drop les données résiduelles
df_test_na_pib=df_test_na_pib.drop(['Population_x'],axis=1)
df_test_na_pib=df_test_na_pib.drop(['PIB/habitant_x'],axis=1)
df_test_na_pib=df_test_na_pib.drop(['PIB_x'],axis=1)

# On renomme les colonnes merge
df_test_na_pib=df_test_na_pib.rename(columns={'Population_y':'Population'})
df_test_na_pib=df_test_na_pib.rename(columns={'PIB/habitant_y':'PIB/habitant'})
df_test_na_pib=df_test_na_pib.rename(columns={'PIB_y':'PIB'})

df_complete=pd.concat([df_complete,df_test_na_pib],ignore_index=True)

In [ ]:
# Attribution du PIB aux pays manquants
df_complete['PIB']=df_complete['PIB'].fillna(df_complete['Country'].map(dictio_pop_gpd))

### Opérations sur les âges

In [75]:
# On attribue les âges manquants en faisant la moyenne des âges par sport
table_sport_age=df_complete.groupby('Sport')['Age'].mean().apply(math.ceil).reset_index()

liste_sport=table_sport_age['Sport'].tolist()
liste_age=table_sport_age['Age'].tolist()

dictio_age_sport=dict(zip(liste_sport,liste_age))

df_complete['Age']=df_complete['Age'].fillna(df_complete['Sport'].map(dictio_age_sport))

In [75] used 12.1 MiB RAM in 0.16s (system mean cpu 36%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 450.6 MiB


In [ ]:
# Il y a 156 personnes qui manquent car nous n'avons pas les sports 
df_complete=df_complete.dropna(subset='Age')

### Opérations sur le sexe, le poids et les tailles

In [73]:
df_male = df_complete[df_complete['Sex'] == 'M']
df_female = df_complete[df_complete['Sex'] == 'F']
columns_to_impute = ['Age', 'Height', 'Weight']

In [73] used 92.4 MiB RAM in 0.22s (system mean cpu 35%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 273.6 MiB


In [74]:
# Créer un imputer KNN avec un nombre de voisins à considérer (k)
imputer = KNNImputer(n_neighbors=5)

# Imputer les valeurs manquantes pour les hommes
df_male[columns_to_impute] = imputer.fit_transform(df_male[columns_to_impute])

# Imputer les valeurs manquantes pour les femmes
df_female[columns_to_impute] = imputer.fit_transform(df_female[columns_to_impute])

# Fusionner les DataFrames pour obtenir le DataFrame complet
df_complete = pd.concat([df_male, df_female])

/var/folders/38/1vcxjcxx0z75mdh8whh4pw6r0000gn/T/ipykernel_59313/3237411982.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_male[columns_to_impute] = imputer.fit_transform(df_male[columns_to_impute])


In [74] used 151.8 MiB RAM in 1025.85s (system mean cpu 54%, single max cpu 100%), peaked 3296.2 MiB above final usage, current RAM usage now 425.4 MiB


/var/folders/38/1vcxjcxx0z75mdh8whh4pw6r0000gn/T/ipykernel_59313/3237411982.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_female[columns_to_impute] = imputer.fit_transform(df_female[columns_to_impute])


## Traitement des valeurs aberrantes

In [ ]:
# A faire
# TODO : Guillaume

## Encodage des données

Encodage des médailles

In [47]:
# Encodage du sexe
df_complete['Sex'].replace(['F','M'],['0', '1'],inplace=True)

# Encodage des médailles 
df_complete['Medal'].replace(['Gold','Silver','Bronze'],['5', '3','1'],inplace=True)

# transform the medal into numeric values
df_complete['Medal'] = df_complete.Medal.fillna('0')

#JO Hivers = 0; été = 1
df_complete['Season'].replace(['Winter','Summer'],['0', '1'],inplace=True)
df_complete = df_complete.drop(df_complete[df_complete['Season'] == '0'].index)

# Encodage de l'âge
df_complete['Age'] = pd.cut(df_complete.Age, range(0, 80, 5), right=False)

# Encodage du poids
df_complete['Weight'] = pd.cut(df_complete.Weight, range(0, 230, 10), right=False)

# Encodage de la taille
df_complete['Height'] = pd.cut(df_complete.Height, range(0, 250, 10), right=False)

# Encodage du sport 
encoder=preprocessing.OneHotEncoder(sparse_output=False)
encoder.fit(df_complete[['Sport']])
columns_name = encoder.get_feature_names_out(['Sport'])
df_encoded = pd.DataFrame(encoder.transform(df_complete[['Sport']]))
df_encoded.columns = columns_name
df_complete= pd.concat([df_complete, df_encoded], axis=1)

df_complete = df_complete.drop('Season', axis=1)

In [47] used -404.9 MiB RAM in 0.11s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 20.8 MiB


### Attribution score et poids médaille (Code Noé )

### Nettoyage des colonnes inutiles pour l'analyse

In [ ]:
# Drop les colonnes qui ne servent à R

## Traitement des valeurs manquantes 

Effectivement notre base de données possède de nombreuses valeurs manquantes 

In [ ]:
# df.isna().sum()

In [ ]:
### Première solution : Web Scrapping (code Nathan + FLO)

# Data understanding

## EDA (graph, pivot table ,Matrix confusion...)

In [3]:
# TODO: Guillaume / Noé / Flo

## Tests de corrélations/lois stats (Shapiro-Wilk, Spearman, Pearson)

In [ ]:
# TODO: Noé